In [ ]:
# import cv2
# import numpy as np

# # Load the image
# image = cv2.imread('just_blocks.jpg')

# # Convert the image to grayscale
# gray = cv2.GaussianBlur(image, (9, 9), 0)
# gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

# # Adjustable parameters
# quality_level = 0.10  # You can experiment with this value
# min_distance = 1  # Adjust this value based on the distance between corners
# block_size = 2  # You can experiment with this value

# # Perform Shi-Tomasi corner detection with adjusted parameters
# corners = cv2.goodFeaturesToTrack(gray, maxCorners=100, qualityLevel=quality_level,
#                                    minDistance=min_distance, blockSize=block_size)

# # Convert corners to integers
# corners = np.int0(corners)

# # Draw circles at detected corners
# for corner in corners:
#     x, y = corner.ravel()
#     cv2.circle(image, (x, y), 5, 255, -1)

# # Extract center coordinates of the detected corners
# center_coordinates = [(corner[0][0], corner[0][1]) for corner in corners]

# # Find the centroid of each object
# centroids = []
# for i in range(0, len(center_coordinates), 4):  # Assuming 4 corners per object
#     object_points = center_coordinates[i:i+4]
#     centroid = (sum(x for x, y in object_points) // 4, sum(y for x, y in object_points) // 4)
#     centroids.append(centroid)
#     cv2.circle(image, centroid, 10, (0, 255, 0), -1)  # Plot the centroid on the image

# # Save the image with detected corners and centroids
# output_path = 'imgs/result_image_with_corners_and_centroids.jpg'
# cv2.imwrite(output_path, image)

# # Print the center coordinates
# print("Centroid Coordinates:")
# for centroid in centroids:
#     print(centroid)


In [41]:
import cv2
import numpy as np
from math import atan2, cos, sin, sqrt, pi
import numpy as np


def drawAxis(img, p_, q_, color, scale):
  p = list(p_)
  q = list(q_)
 
  ## [visualization1]
  angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
  hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
 
  # Here we lengthen the arrow by a factor of scale
  q[0] = p[0] - scale * hypotenuse * cos(angle)
  q[1] = p[1] - scale * hypotenuse * sin(angle)
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)
 
  # create the arrow hooks
  p[0] = q[0] + 9 * cos(angle + pi / 4)
  p[1] = q[1] + 9 * sin(angle + pi / 4)
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)
 
  p[0] = q[0] + 9 * cos(angle - pi / 4)
  p[1] = q[1] + 9 * sin(angle - pi / 4)
  cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), color, 3, cv2.LINE_AA)
  ## [visualization1]
 
def getOrientation(pts, img):
  ## [pca]
  # Construct a buffer used by the pca analysis
  sz = len(pts)
  data_pts = np.empty((sz, 2), dtype=np.float64)
  for i in range(data_pts.shape[0]):
    data_pts[i,0] = pts[i,0,0]
    data_pts[i,1] = pts[i,0,1]
 
  # Perform PCA analysis
  mean = np.empty((0))
  mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
 
  # Store the center of the object
  cntr = (int(mean[0,0]), int(mean[0,1]))
  ## [pca]
 
  ## [visualization]
  # Draw the principal components
  cv2.circle(img, cntr, 3, (255, 0, 255), 2)
  p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
  p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
  drawAxis(img, cntr, p1, (255, 255, 0), 1)
  drawAxis(img, cntr, p2, (0, 0, 255), 5)
 
  angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
  ## [visualization]
 
  # Label with the rotation angle
  label = "  Rotation Angle: " + str(-int(np.rad2deg(angle)) - 90) + " degrees"
  textbox = cv2.rectangle(img, (cntr[0], cntr[1]-25), (cntr[0] + 250, cntr[1] + 10), (255,255,255), -1)
  cv2.putText(img, label, (cntr[0], cntr[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
 
  return angle








def detect_boxes_in_area(image_path, output_path="/home/jp/Desktop/visionSort/visionSort_cv/imgs/detected_blocks.jpg"):
    # Read the image
    img = cv2.imread(image_path)

    center_x, center_y = img.shape[1] // 2, img.shape[0] // 2


    area_width, area_height = 855, 1050

    roi = img[center_y - area_height // 2: center_y + area_height // 2,
              center_x - area_width // 2: center_x + area_width // 2]

    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray_roi, (5, 5), 0)

    edges = cv2.Canny(blurred, 5, 200)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 20]

    for i, cnt in enumerate(contours):
        x, y, w, h = cv2.boundingRect(cnt)
        
        # Adjust the coordinates based on the ROI position
        x += center_x - area_width // 2
        y += center_y - area_height // 2

        # Calculate the center coordinates
        center_x_box = x + w // 2
        center_y_box = y + h // 2

        # Draw the contours
        cv2.drawContours(img, [np.array([[x, y], [x+w, y], [x+w, y+h], [x, y+h]])], -1, (0, 255, 0), 2)
        
        # Draw the center on the image
        cv2.circle(img, (center_x_box, center_y_box), 5, (255, 255, 255), -1)
        
        # Print information
        cv2.putText(img, f"Box {i + 1}: ({x}, {y})", (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(img, f"Center {i + 1}: ({center_x_box}, {center_y_box})", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        print(f"Box {i + 1} - Coordinates: ({x}, {y}), ({x + w}, {y + h}), Center: ({center_x_box}, {center_y_box})")
        
        getOrientation(cnt, img)
    # Save the result as an image
    cv2.imwrite(output_path, img)
    print(f"Result saved at: {output_path}")

# Provide the path to your image file
image_path = '/home/jp/Desktop/visionSort/visionSort_cv/more_blocks.jpg'
detect_boxes_in_area(image_path)


Box 1 - Coordinates: (844, 1123), (949, 1325), Center: (896, 1224)
Box 2 - Coordinates: (711, 1075), (776, 1155), Center: (743, 1115)
Box 3 - Coordinates: (647, 1066), (773, 1238), Center: (710, 1152)
Box 4 - Coordinates: (758, 1038), (942, 1199), Center: (850, 1118)
Box 5 - Coordinates: (639, 1029), (959, 1325), Center: (799, 1177)
Box 6 - Coordinates: (344, 939), (572, 1185), Center: (458, 1062)
Box 7 - Coordinates: (692, 729), (935, 952), Center: (813, 840)
Box 8 - Coordinates: (597, 624), (709, 670), Center: (653, 647)
Box 9 - Coordinates: (472, 580), (491, 593), Center: (481, 586)
Box 10 - Coordinates: (517, 495), (533, 512), Center: (525, 503)
Box 11 - Coordinates: (642, 489), (670, 519), Center: (656, 504)
Box 12 - Coordinates: (507, 486), (550, 514), Center: (528, 500)
Box 13 - Coordinates: (622, 478), (673, 530), Center: (647, 504)
Box 14 - Coordinates: (588, 477), (603, 492), Center: (595, 484)
Box 15 - Coordinates: (545, 468), (681, 537), Center: (613, 502)
Box 16 - Coordina